### 混合ガウスモデルを使ったクラスタリングの実装
https://qiita.com/isuya/items/018a0867bdc95033736d

#### アヤメのデータセット

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
df = sns.load_dataset('iris')
ax1.scatter(df['petal_length'], df['petal_width'], color='gray')
ax1.set_title('without label')
ax1.set_xlabel('petal_length')
ax1.set_ylabel('petal_width')

for sp in df['species'].unique():
  x = df[df['species'] == sp]['petal_length']
  y = df[df['species'] == sp]['petal_width']
  ax2.scatter(x, y, label=sp)

ax2.legend()
ax2.set_title('with label')
ax2.set_xlabel('petal_length')
ax2.set_ylabel('petal_width')
plt.show()

#### クラスタのデータ数をカウントできる配列の実装

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
class ClusterArray(object):
  def __init__(self, array):
    # arrayは1次元のリスト、配列
    self._array = np.array(array, dtype=np.int)
    self._counter = Counter(array)

  @property
  def array(self):
    return self._array.copy()

  def count(self, k):
    return self._counter[k]

  def __setitem__(self, i, k):
    # 実行されるとself._counterも更新される
    pre_value = self._array[i]
    if pre_value == k:
      return

    if self._counter[pre_value] > 0:
      self._counter[pre_value] -= 1
    self._array[i] = k
    self._counter[k] += 1

  def __getitem__(self, i):
    return self._array[i]

#### 余分な計算の削除

In [ ]:
def log_deformed_gaussian(x, mu, var):
  norm_squared = ((x - mu) * (x - mu)).sum(axis=1)
  return -norm_squared / (2 * var)

#### 実装

In [ ]:
class GaussianMixtureClustering(object):
  def __init__(self, K, D, var=1, var_pri=1, seed=None):
    self.K = K  # クラスタ数
    self.D = D  # 説明変数の次元(実装しやすたのため、コンストラクタの時点で設定しておく)
    self.z = None

    # 確率分布のパラメータ設定
    self.mu = np.zeros((self.K, self.D))
    self.var = var  # 固定、すべてのクラスタで共通
    self.pi = np.full(self.K, 1 / self.K)  # 固定、すべてのクラスタで共通

    # 事前分布の設定
    self.mu_pri = np.zeros(self.D)
    self.var_pri = var_pri

    self._random = np.random.RandomState(seed)

  def fit(self, X, n_iter):
    init_z = self._random.randint(0, self.K, X.shape[0])
    self.z = ClusterArray(init_z)

    for _ in range(n_iter):
      for k in range(self.K):
        self.mu[k] = self._sample_mu_k(X, k)
      for i, x_i in enumerate(X):
        self.z[i] = self._sample_zi(x_i)

  def _sample_zi(self, x_i):
    log_probs_xi = log_deformed_gaussian(x_i, self.mu, self.var)

    probs_zi = np.exp(log_probs_xi) * self.pi
    probs_zi = probs_zi / probs_zi.sum()

    z_i = self._random.multinomial(1, probs_zi)
    z_i = np.where(z_i)[0][0]
    return z_i

  def _sample_mu_k(self, X, k):
    xk_bar = np.array([x for i, x in enumerate(X) if self.z[i] == k]).mean(axis=0)
    var_pos = 1 / (self.z.count(k) / self.var + 1 / self.var_pri)
    mu_pos = var_pos * (xk_bar * self.z.count(k) / self.var + self.mu_pri / self.var_pri)

    mu_k = self._random.multivariate_normal(mu_pos, var_pos * np.eye(self.D))
    return mu_k

#### クラスタリングを試してみる

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# データセットの読み込み
df = sns.load_dataset('iris')
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values

# 混合ガウスモデルによるクラスタリング
gmc = GaussianMixtureClustering(K=3, D=4, var=0.1, seed=1)
gmc.fit(X, n_iter=10)
df['GMM_cluster'] = gmc.z.array

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
for sp in df['species'].unique():
  x = df[df['species'] == sp]['petal_length']
  y = df[df['species'] == sp]['petal_width']
  ax1.scatter(x, y, label=sp)

ax1.legend()
ax1.set_title('species')
ax1.set_xlabel('petal_length')
ax1.set_ylabel('petal_width')

for k in range(gmc.K):
  x = df[df['GMM_cluster'] == k]['petal_length']
  y = df[df['GMM_cluster'] == k]['petal_width']
  ax2.scatter(x, y, label=k)

ax2.legend()
ax2.set_title('GMM cluster')
ax2.set_xlabel('petal_length')
ax2.set_ylabel('petal_width')
plt.show()

In [ ]:
sns.pairplot(
    df.drop(columns=['species']),
    vars=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
    hue='GMM_cluster'
    )

### pythonで混合正規分布実装
https://qiita.com/ta-ka/items/3e8b127620ac92a32864

In [ ]:
import numpy
from matplotlib import pyplot
import sys

In [ ]:
def create_data(N, K):
  X, mu_star, sigma_star = [], [], []
  for i in range(K):
    loc = (numpy.random.rand() - 0.5) * 10.0 # range: -5.0 - 5.0
    scale = numpy.random.rand() * 3.0 # range: 0.0 - 3.0
    X = numpy.append(X, numpy.random.normal(loc = loc, scale = scale, size = int(N / K)))
    mu_star = numpy.append(mu_star, loc)
    sigma_star = numpy.append(sigma_star, scale)
  return (X, mu_star, sigma_star)

In [ ]:
def gaussian(mu, sigma):
  def f(x):
    return numpy.exp(-0.5 * (x - mu) ** 2 / sigma) / numpy.sqrt(2 * numpy.pi * sigma)
  return f

In [ ]:
def estimate_posterior_likelihood(X, pi, gf):
  l = numpy.zeros((X.size, pi.size))
  for (i, x) in enumerate(X):
    l[i, :] = gf(x)
  return pi * l * numpy.vectorize(lambda y: 1 / y)(l.sum(axis = 1).reshape(-1, 1))

In [ ]:
def estimate_gmm_parameter(X, gamma):
  N = gamma.sum(axis = 0)
  mu = (gamma * X.reshape((-1, 1))).sum(axis = 0) / N
  sigma = (gamma * (X.reshape(-1, 1) - mu) ** 2).sum(axis = 0) / N
  pi = N / X.size
  return (mu, sigma, pi)

In [ ]:
def calc_Q(X, mu, sigma, pi, gamma):
  Q = (gamma * (numpy.log(pi * (2 * numpy.pi * sigma) ** (-0.5)))).sum()
  for (i, x) in enumerate(X):
    Q += (gamma[i, :] * (-0.5 * (x - mu) ** 2 / sigma)).sum()
  return Q

In [ ]:
if __name__ == '__main__':

  # data
  K = 2
  N = 1000 * K
  X, mu_star, sigma_star = create_data(N, K)

  # termination condition
  epsilon = 0.000001

  # initialize gmm parameter
  pi = numpy.random.rand(K)
  mu = numpy.random.randn(K)
  sigma = numpy.abs(numpy.random.randn(K))
  Q = -sys.float_info.max
  delta = None

  # EM algorithm
  while delta == None or delta >= epsilon:
    gf = gaussian(mu, sigma)

    # E step: estimate posterior probability of hidden variable gamma
    gamma = estimate_posterior_likelihood(X, pi, gf)

    # M step: miximize Q function by estimating mu, sigma and pi
    mu, sigma, pi = estimate_gmm_parameter(X, gamma)

    # calculate Q function
    Q_new = calc_Q(X, mu, sigma, pi, gamma)
    delta = Q_new - Q
    Q = Q_new

  # result
  print(u'mu*: %s, simga*: %s' % (str(numpy.sort(numpy.around(mu_star, 3))), str(numpy.sort(numpy.around(sigma_star, 3)))))
  print(u'mu : %s, sgima : %s' % (str(numpy.sort(numpy.around(mu, 3))), str(numpy.sort(numpy.around(sigma, 3)))))

  # plot
  n, bins, _ = pyplot.hist(X, 50, density=True, alpha = 0.3)
  seq = numpy.arange(-10, 10, 0.02)
  for i in range(K):
    pyplot.plot(seq, gaussian(mu[i], sigma[i])(seq), linewidth = 2.0)

  pyplot.savefig('gmm_graph.png')
  pyplot.show()

### ガウス混合分布のパラメータをscikit-learnで推定する
https://ohke.hateblo.jp/entry/2019/06/08/230000

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_iris

In [ ]:
# irisデータセットのロード
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target

# 種類 (ラベル) によって、サンプル数を変えます
d1 = df[df['label'] == 0].sample(30)  # setosa
d2 = df[df['label'] == 1].sample(50)  # versicolor
d3 = df[df['label'] == 2].sample(40)  # virginica

# 萼片長のデータのみを使う
X = pd.concat([d1['sepal length (cm)'], d2['sepal length (cm)'], d3['sepal length (cm)']])
Y = pd.concat([d1['label'], d2['label'], d3['label']])

# ヒストグラム
plt.hist([X[Y==0], X[Y==1], X[Y==2]], bins=np.arange(X.min(), X.max(), 0.2), stacked=True, label=iris.target_names)
plt.legend()
plt.plot()

In [ ]:
# GaussianMixtureの学習
gmm = GaussianMixture(
    n_components=3,
    covariance_type='spherical'
    ).fit(
        np.array(X).reshape(-1, 1)  # 次元数2を入力とするため変形
        )

# 重み
print(gmm.weights_)
# [0.309359   0.43752389 0.25311711]

# 期待値
print(gmm.means_)
# [[5.80722255]
#  [6.57885203]
#  [4.93215112]]

# 分散
print(gmm.covariances_)
# [0.10564111 0.32196481 0.06607687]

In [ ]:
import scipy

x = np.linspace(3, 9, 600)

gd1 = scipy.stats.norm.pdf(x, gmm.means_[0, -1], np.sqrt(gmm.covariances_[0]))
gd2 = scipy.stats.norm.pdf(x, gmm.means_[1, -1], np.sqrt(gmm.covariances_[1]))
gd3 = scipy.stats.norm.pdf(x, gmm.means_[2, -1], np.sqrt(gmm.covariances_[2]))
    
plt.plot(x, gmm.weights_[0] * gd1, label='gd1')
plt.plot(x, gmm.weights_[1] * gd2, label='gd2')
plt.plot(x, gmm.weights_[2] * gd3, label='gd3')
plt.legend()
plt.show()

In [ ]:
# 属する分布を予測
Y_predict = gmm.predict(np.array(X).reshape(-1, 1))
print(Y_predict)
# [2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 0 2 0 2 2 2 0 2 2 2 2 2 2 2 2 0 0 2 1 0 1 0
#  0 0 0 0 2 0 0 1 0 2 1 0 1 0 1 0 1 0 0 0 1 0 0 1 2 0 0 0 1 1 0 0 1 1 0 0 0
#  0 0 1 1 2 1 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 0 2 1 1 1 1 1 1 1 1 0 1 1
#  1 1 1 1 1 1 1 1 1]

# ラベルを 0->2, 1->0, 2->1 へ置き換える
Y_new = Y.copy()
Y_new[Y==0] = 2
Y_new[Y==1] = 0
Y_new[Y==2] = 1

# 精度を計算
print(sum(Y_new == Y_predict) / len(Y_new))
# 0.7166666666666667

### 混合ガウスモデル
https://mukai-lab.info/pages/classes/intelligence_information_system/chapter12/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.random.normal(0, 1, 10000) # 10000の乱数を生成

plt.hist(x, bins=100) # 階級数を100に設定したヒストグラム
plt.xlim(-5, 5)
plt.xlabel("x")
plt.ylabel("Frequency")


In [ ]:
x1 = np.random.normal(167.7,  6.6, 10000) 
x2 = np.random.normal(178.4, 7.6, 10000)
x = np.concatenate((x1, x2)) # 結合（重ね合わせ）

plt.hist(x, bins=100)
plt.xlabel("x")
plt.ylabel("Frequency")

In [ ]:
x = np.array(x).reshape(-1, 1)
print(x)

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(
    n_components=2, # 重ね合わせた分布数
	  covariance_type="spherical" # 分散は共通（球状）
	  ).fit(x)

In [ ]:
weight = gmm.weights_ #重み（足すと1になる）
mean = gmm.means_ #平均
sd = np.sqrt(gmm.covariances_) #標準偏差
print(weight)
print(mean)
print(sd)

In [ ]:
from scipy.stats import norm

x_ = np.arange(140, 210, 0.1)
y1 = weight[0] * norm.pdf(x_, mean[0], sd[0])
y2 = weight[1] * norm.pdf(x_,mean[1], sd[1])

plt.plot(x_, y1)
plt.plot(x_, y2)
plt.xlabel("Height")
plt.ylabel("Probability Density")

In [ ]:
y3 = y1 + y2

plt.plot(x_, y3)
plt.xlabel("Height")
plt.ylabel("Probability Density")

In [ ]:
sample1 = np.array([[165]]) # 165cm
sample2 = np.array([[185]]) # 185cm

result1 = gmm.predict(sample1)
result2 = gmm.predict(sample2)

print(result1) # 日本人
print(result2) # アメリカ人

### pythonのscikit-learnで混合ガウスモデルを用いたクラスタリング
https://akitoshiblogsite.com/python-scikit-learn-gmm/

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns
sns.set(context="paper" , style ="whitegrid",rc={"figure.facecolor":"white"})

from scipy import linalg
import itertools
# for Gaussian Mixture model for classification
from sklearn import mixture
# for PCA (principal component analysis)
from sklearn.decomposition import PCA

# for sample data
from sklearn import datasets

In [ ]:
dataset = datasets.load_iris()
dic = {}
for k,v in zip(dataset.feature_names,dataset.data.T):
  dic[k] = v

df = pd.DataFrame(dic)
df.head().T

In [ ]:
X = df.values
lowestBIC = np.infty
bic = []
nComponentsRange = range(1, 7)
cvTypes = ['spherical', 'tied', 'diag', 'full']
nInit = 10
for cvType in cvTypes:
  for nComponents in nComponentsRange:
    # Fit a Gaussian mixture with EM
    gmm = mixture.GaussianMixture(n_components=nComponents,
                                  covariance_type=cvType,n_init =nInit)
    gmm.fit(X)
    bic.append(gmm.bic(X))
    if bic[-1] < lowestBIC:
      lowestBIC = bic[-1]
      bestGmm = gmm
            
bic = np.array(bic)
colorIter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                             'darkorange'])

bars = []

# Plot the BIC scores
plt.figure(figsize=(8, 6),dpi=100)
ax = plt.subplot(111)
for i, (cvType, color) in enumerate(zip(cvTypes, colorIter)):
  xpos = np.array(nComponentsRange) + .2 * (i - 2)
  bars.append(plt.bar(xpos,
                      bic[i * len(nComponentsRange):(i + 1) * len(nComponentsRange)],
                      width=.2, color=color))

plt.xticks(nComponentsRange)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(),
              len(nComponentsRange)) + .65 + .2 * np.floor(bic.argmin() / len(nComponentsRange)
              )

plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
ax.set_xlabel('Number of components')
ax.legend([b[0] for b in bars], cvTypes)

In [ ]:
def summaryPCA(pca_,cols):
  print('各次元の寄与率: {0}'.format(pca_.explained_variance_ratio_))
  print('累積寄与率: {0}'.format(sum(pca_.explained_variance_ratio_)))
  print("pca components")
  for k,v in zip(cols,pca_.components_.T):
    print(k,":",", ".join(["%2.2f"%i for i in v]))

In [ ]:
def contriPCAPlot(pca_,cols):
  c_p = pca_.components_[:2]

  fig = plt.figure(figsize=(4,4),dpi=100)
  ax = fig.add_subplot(111)
  ax.set_xlim([np.min(c_p[0] + [0])-0.1,np.max(c_p[0]) + 0.1])
  ax.set_ylim([np.min(c_p[1]+[0])-0.1,np.max(c_p[1])+0.1])
  # colormap setting
  cm = plt.get_cmap("hsv")
  c = []
  n_ = len(c_p[0])
  for i in range(n_):
    c.append(cm(i/n_))
        
  for i, v in enumerate(c_p.T):
    ax.arrow(x=0,y=0,dx=v[0],dy=v[1],width=0.01,head_width=0.05,
                  head_length=0.05,length_includes_head=True,color=c[i])
  # legend setting 
  patch = []
  for i in range(n_):
    patch.append(mpatches.Patch(color=c[i], label=cols[i]))

  plt.legend(handles=patch,bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=8)
  ax.set_title("pca_components_ for each variable")
  plt.show()

In [ ]:
def clusterPCAPlot(trans_,df_):
  fig = plt.figure(figsize=(6,4),dpi=100)
  ax = fig.add_subplot(111)
  for i in range(len(df_["clusterId"].unique())): 
    ax.scatter(trans_[df_["clusterId"]==i,0],trans_[df_["clusterId"]==i,1])

  plt.legend(df_["clusterId"].unique())
  plt.show()

In [ ]:
varNames = df.columns
# normalization
dfScaled = df.copy()[varNames]
dfScaled = (dfScaled - dfScaled.mean())/dfScaled.std()

# principal components analysis
pca = PCA(n_components=3)
trans= pca.fit_transform(dfScaled[varNames])

# insert results of GMM clustring 
dfScaled["clusterId"] = bestGmm.predict(X)

# these functions are defined in the previous article. 
summaryPCA(pca,varNames)
contriPCAPlot(pca,varNames)
clusterPCAPlot(trans,dfScaled)

### EMアルゴリズムのpythonによる実装と一般化
https://masamunetogetoge.com/em-algorithm-python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
x1 = np.random.normal(loc=0.5, scale=1 , size =100).reshape(-1,1)
x2 = np.random.normal(loc=10, scale=2 , size =100).reshape(-1,1)
x3 = np.random.normal(loc=0, scale=3 , size =100).reshape(-1,1)
x = np.concatenate([x1 , x2 , x3])
sns.displot( x )
plt.title("Gaussian Mixture Model")

In [ ]:
def Gaus(x,m, s): #正規分布の値を取得
  g = np.exp( - pow((x-m), 2) /(2*s) )/np.sqrt(2*np.pi * s )
  return g
 
def calc_gamma(x, pi, mu, sig):#事後分布の計算
  gam = pi*Gaus(x,mu,sig)
  gam/= np.sum(gam, axis=1).reshape(len(x),1)
  return gam

def update_parmas(gamma, x, pi, mu, sig):#パラメーターの更新式
  N_k = np.sum(gamma, axis=0)
  N = np.sum(N_k)
  mu_k = np.sum(x*gamma, axis=0 ) /N_k
  sig_k = np.sum(gamma* pow(x-mu, 2), axis=0) /N_k
  pi_k = N_k/N
  return pi_k , mu_k, sig_k

def iteration(x,mu,sig,pi, I=100, e=0.01): #ε以下になるか、100回計算するまで尤度を更新する関数
  LF=0   
  for i in range(I):
    gamma = calc_gamma(x, pi, mu, sig)
    LF_new =np.sum(np.log(np.sum(pi*Gaus(x,mu,sig),axis=1 )) )
    ch = LF_new - LF
    print("LF ={} . change = {}".format(LF_new, ch))
    if np.abs(ch) < e:
      print("Iteration is finished {} iter. ".format(i+1))
      break
    LF=LF_new
    pi, mu, sig = update_parmas(gamma, x, pi, mu, sig)
 
  return pi, mu, sig   

In [ ]:
mu =np.array([0,10,3])
sig=np.array([1, 5, 10])
pi=np.array([0.1,0.4, 0.5])
pi, mu, sig = iteration(x,mu,sig, pi, I=100)

print(pi, mu, np.sqrt(sig) )

In [ ]:
y0 = np.random.normal(loc=mu[0], scale=np.sqrt(sig)[0] , size =int(300*pi[0]) ).reshape(-1,1)
y1 = np.random.normal(loc=mu[1], scale=np.sqrt(sig)[1] , size =int(300*pi[1]) ).reshape(-1,1)
y2 = np.random.normal(loc=mu[2], scale=np.sqrt(sig)[2] , size =int(300*pi[2]) ).reshape(-1,1)
y=np.concatenate([y0, y1, y2])
sns.displot(y)
plt.title("Predicted GMM")

### EMアルゴリズム解説とPythonによるGMM（混合ガウス分布）への実装
https://tips-memo.com/python-emalgorithm-gmm

In [ ]:
import sys
import csv
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
txt_dir = sys.argv[1]
csv_name = sys.argv[2]
dat_name = sys.argv[3]

In [ ]:
with open(txt_dir) as f:
    reader = csv.reader(f)
    l = [row for row in reader]
    for i in range(len(l)):
      for j in range(len(l[i])):
        l[i][j] = float(l[i][j])
l = np.array(l)

In [ ]:
def calc(x, mu, sigma_inv, sigma_det):
  D = x.shape[0]
  exp = -0.5*(x - mu).T@sigma_inv.T@(x - mu)
  denomin = np.sqrt(sigma_det)*(np.sqrt(2*np.pi)**D)
  return np.exp(exp)/denomin

In [ ]:
def gauss(X, mu, sigma):
  output = np.array([])
  eps = np.spacing(1)
  Eps = eps*np.eye(sigma.shape[0])
  sigma_inv = la.inv(sigma)
  sigma_det = la.det(sigma)
  N = X.shape[0]
  for i in range(N):
    output = np.append(output, calc(X[i], mu, sigma_inv, sigma_det))
  return output

In [ ]:
def mix_gauss(X, Mu, Sigma, Pi):
  k = len(Mu)
  output = np.array([Pi[i]*gauss(X, Mu[i], Sigma[i]) for i in range(k)])
  return output, np.sum(output, 0)[None,:]

In [ ]:
def setInitial(X, k):
  D = X.shape[1]
  Mu = np.random.randn(k, D)
  Sigma = np.array([np.eye(D) for i in range(k)])
  Pi = np.array([1/k for i in range(k)])
  return Mu, Sigma, Pi

In [ ]:
def log_likelihood(X, Mu, Sigma, Pi):
  K = Mu.shape[0]
  D = X.shape[1]
  N = X.shape[0]
  _, out_sum = mix_gauss(X, Mu, Sigma, Pi)
  logs = np.array([np.log(out_sum[0][n]) for n in range(N)])
  return np.sum(logs)

In [ ]:
def EM(X, k, Mu, Sigma, Pi, thr):
  K = Mu.shape[0]
  D = X.shape[1]
  N = X.shape[0]
  log_list = np.array([])
  log_list = np.append(log_list, log_likelihood(X, Mu, Sigma, Pi))
  count = 0
  while True:
    out_com, out_sum = mix_gauss(X, Mu, Sigma, Pi)
    gamma = out_com / out_sum
    N_k = np.sum(gamma, 1)[:,None]
    Mu = (gamma @ X) / N_k

    sigma_list = np.zeros((N, K, D, D))
    for k in range(K):
      for n in range(N):
        sigma_com = gamma[k][n]*(X[n] - Mu[k])[:,None]@((X[n] - Mu[k]))[None,:]
        sigma_list[n][k] = sigma_com
    Sigma = np.sum(sigma_list, 0) / N_k[:,None]
    Pi = N_k/N
    
    log_list = np.append(log_list, log_likelihood(X, Mu, Sigma, Pi))
    if np.abs(log_list[count] - log_list[count+1]) < thr:
      return count+1, log_list, Mu, Sigma, Pi, gamma
    else:
      print("Previous log-likelihood gap:" + str(np.abs(log_list[count] - log_list[count+1])))
      count += 1

In [ ]:
thr = 0.01
k = 4
Mu, Sigma, Pi = setInitial(l, k)
n_iter, log_list, Mu, Sigma, Pi, gamma = EM(l, k, Mu, Sigma, Pi, thr)
print("Iteration:"+str(n_iter))
print("log-likelihood:"+str(log_list))

params = np.array([Mu.ravel(), Sigma.ravel(), Pi.ravel()])
index = np.argmax(gamma, 0)

cm = plt.get_cmap("tab10")
fig = plt.figure()
ax = Axes3D(fig)
N = l.shape[0]


In [ ]:
for n in range(N):
  ax.plot([l[n][0]], [l[n][1]], [l[n][2]],  "o", color=cm(index[n]), ms=1.5)
ax.view_init(elev=30, azim=45)
plt.show()

with open(csv_name, 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(gamma.T)

with open(dat_name, 'w') as file:
    writer = csv.writer(file, lineterminator='\n\n')
    writer.writerows(params)

### 混合ガウス分布でクラスタリング
https://openbook4.me/projects/231/sections/1549


#### 記事が古すぎてコードが動かない

自力で修正できず。。。

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [ ]:
# 二次元正規分布に従うサンプルをランダムに生成

N = 300
x = np.concatenate([np.random.multivariate_normal([-2, 0], np.eye(2), int(N / 3)),
                    np.random.multivariate_normal([0, 5], np.eye(2), int(N / 3)),
                    np.random.multivariate_normal([2, 3], np.eye(2), int(N / 3))]
                   )

In [ ]:
# 混合ガウス分布で教師なし学習

K = 3  #クラスタ数は3とする
gmm = GaussianMixture(n_components=K)
gmm.fit(x) #学習

In [ ]:
plt.xlim(-5, 5)
plt.ylim(-3, 8)
#gmm.predictで予想したクラスを返す   cはcolor, sは点の大きさ
plt.scatter(x[:, 0], x[:, 1], c = gmm.predict(x) , s = 50)
X, Y = np.meshgrid(np.linspace(-5, 5), np.linspace(-3, 8))
 
#gmm.scoreでlog probabilityを返す
#gmm.predict_probaは事後確率なのでここでは使わない
f = lambda x, y: np.exp(gmm.score(np.array([x, y])))
 
#X,Yのそれぞれの要素ごとに計算し、同じ大きさの配列を返す
#例えば、X[4,6],Y[4,6]の要素をfに代入し、Z[4,6]とする
Z = np.vectorize(f)(X, Y)
 
#alphaは透明度
plt.pcolor(X, Y, Z, alpha = 0.2)
 
#あるとｚ軸の値がわかって便利
plt.colorbar()
 
plt.show()

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

N = 150
x = np.concatenate([np.random.normal(0, 0.3, int(N / 3)),
                    np.random.normal(2, 0.5, int(N / 3)),
                    np.random.normal(5, 0.2, int(N / 3))]
                   )

#3クラスとする
K = 3
x = list(map(lambda x: [x], x))   #なぜか[[1],[2],[3],...]の形にする必要がある

gmm = GaussianMixture(n_components=K)
gmm.fit(x) #学習
 
plt.xlim(-3, 7)
plt.scatter(x, np.zeros(N), c = gmm.predict(x))
X = np.linspace(-3, 7, 200)
f = lambda x: np.exp(gmm.score(x))
Y = np.vectorize(f)(X)
plt.plot(X,Y)
plt.show()